# Part 1: Data Preparation & Embedding

This notebook implements the first part of the search engine project:
1. Load the dataset
2. Choose a pre-trained multimodal model
3. Generate vector embeddings for images and text
4. Store embeddings with metadata

## Dataset: Flickr8k
We'll use the Flickr8k dataset which contains 8,000 images with 5 captions each.


## 1. Setup and Imports


In [ ]:
# Import necessary libraries
import torch
import torchvision.transforms as transforms
from transformers import CLIPProcessor, CLIPModel
from PIL import Image
import numpy as np
import pandas as pd
import os
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


## 2. Load and Explore Dataset


In [ ]:
# Create data directory structure
os.makedirs('data', exist_ok=True)
os.makedirs('data/images', exist_ok=True)
os.makedirs('embeddings', exist_ok=True)

print("Directory structure created successfully!")


In [ ]:
# For this demo, we'll create a sample dataset structure
# In a real scenario, you would download Flickr8k from the official source

# Create sample captions file (this would normally be downloaded)
sample_captions = [
    "A dog is running in the park",
    "A cat is sitting on a windowsill", 
    "Children are playing in the playground",
    "A beautiful sunset over the ocean",
    "A person riding a bicycle on the street",
    "A bird is flying in the sky",
    "A car is parked in front of a house",
    "A flower garden in full bloom",
    "A mountain landscape with snow",
    "A person cooking in the kitchen"
]

# Create sample captions file
with open('data/captions.txt', 'w') as f:
    for i, caption in enumerate(sample_captions):
        f.write(f"{i+1:04d}.jpg {caption}\n")

print(f"Created sample captions file with {len(sample_captions)} entries")


In [ ]:
# Load captions data
def load_captions(captions_file):
    """Load captions from file"""
    captions = {}
    with open(captions_file, 'r') as f:
        for line in f:
            parts = line.strip().split(' ', 1)
            if len(parts) == 2:
                image_id, caption = parts
                if image_id not in captions:
                    captions[image_id] = []
                captions[image_id].append(caption)
    return captions

# Load our sample captions
captions_data = load_captions('data/captions.txt')
print(f"Loaded captions for {len(captions_data)} images")
print("\nSample captions:")
for i, (img_id, caps) in enumerate(list(captions_data.items())[:3]):
    print(f"{img_id}: {caps[0]}")


## 3. Load Pre-trained Multimodal Model


In [ ]:
# Load CLIP model - a powerful multimodal model from OpenAI
print("Loading CLIP model...")
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name).to(device)
processor = CLIPProcessor.from_pretrained(model_name)

print(f"Model loaded successfully: {model_name}")
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")


## 4. Create Sample Images for Demo


In [ ]:
# Create sample images for demonstration
# In a real scenario, you would have actual images from Flickr8k

from PIL import Image, ImageDraw, ImageFont
import random

def create_sample_image(text, filename, size=(224, 224)):
    """Create a sample image with text for demonstration"""
    # Create a random colored background
    colors = [(255, 200, 200), (200, 255, 200), (200, 200, 255), 
              (255, 255, 200), (255, 200, 255), (200, 255, 255)]
    bg_color = random.choice(colors)
    
    img = Image.new('RGB', size, bg_color)
    draw = ImageDraw.Draw(img)
    
    # Try to use a default font, fallback to basic if not available
    try:
        font = ImageFont.truetype("arial.ttf", 16)
    except:
        font = ImageFont.load_default()
    
    # Draw text in the center
    bbox = draw.textbbox((0, 0), text, font=font)
    text_width = bbox[2] - bbox[0]
    text_height = bbox[3] - bbox[1]
    
    x = (size[0] - text_width) // 2
    y = (size[1] - text_height) // 2
    
    draw.text((x, y), text, fill=(0, 0, 0), font=font)
    
    # Save the image
    img.save(f'data/images/{filename}')
    return img

# Create sample images for each caption
sample_images = []
for i, (img_id, captions) in enumerate(captions_data.items()):
    img = create_sample_image(captions[0], img_id)
    sample_images.append((img_id, captions[0], img))

print(f"Created {len(sample_images)} sample images")


In [ ]:
# Display sample images
fig, axes = plt.subplots(2, 5, figsize=(15, 6))
axes = axes.ravel()

for i, (img_id, caption, img) in enumerate(sample_images[:10]):
    axes[i].imshow(img)
    axes[i].set_title(f"{img_id}\n{caption[:30]}...", fontsize=8)
    axes[i].axis('off')

plt.tight_layout()
plt.show()


## 5. Generate Embeddings


In [ ]:
def get_image_embedding(image_path, model, processor):
    """Generate embedding for an image"""
    try:
        image = Image.open(image_path).convert('RGB')
        inputs = processor(images=image, return_tensors="pt").to(device)
        
        with torch.no_grad():
            image_features = model.get_image_features(**inputs)
            # Normalize the features
            image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        
        return image_features.cpu().numpy().flatten()
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None

def get_text_embedding(text, model, processor):
    """Generate embedding for text"""
    try:
        inputs = processor(text=[text], return_tensors="pt", padding=True, truncation=True).to(device)
        
        with torch.no_grad():
            text_features = model.get_text_features(**inputs)
            # Normalize the features
            text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        
        return text_features.cpu().numpy().flatten()
    except Exception as e:
        print(f"Error processing text '{text}': {e}")
        return None

print("Embedding functions defined successfully!")


In [ ]:
# Generate embeddings for all images and captions
print("Generating embeddings...")

embeddings_data = []

for img_id, captions in tqdm(captions_data.items(), desc="Processing images"):
    image_path = f'data/images/{img_id}'
    
    # Generate image embedding
    image_embedding = get_image_embedding(image_path, model, processor)
    
    if image_embedding is not None:
        # Generate text embeddings for all captions of this image
        for caption in captions:
            text_embedding = get_text_embedding(caption, model, processor)
            
            if text_embedding is not None:
                embeddings_data.append({
                    'image_id': img_id,
                    'image_path': image_path,
                    'caption': caption,
                    'image_embedding': image_embedding,
                    'text_embedding': text_embedding
                })

print(f"Generated embeddings for {len(embeddings_data)} image-caption pairs")


## 6. Store Embeddings


In [ ]:
# Convert embeddings to numpy arrays for easier storage
image_embeddings = np.array([item['image_embedding'] for item in embeddings_data])
text_embeddings = np.array([item['text_embedding'] for item in embeddings_data])

# Create metadata dataframe
metadata = pd.DataFrame([{
    'image_id': item['image_id'],
    'image_path': item['image_path'],
    'caption': item['caption']
} for item in embeddings_data])

print(f"Image embeddings shape: {image_embeddings.shape}")
print(f"Text embeddings shape: {text_embeddings.shape}")
print(f"Metadata shape: {metadata.shape}")

# Display sample metadata
print("\nSample metadata:")
print(metadata.head())


In [ ]:
# Save embeddings and metadata
np.save('embeddings/image_embeddings.npy', image_embeddings)
np.save('embeddings/text_embeddings.npy', text_embeddings)
metadata.to_csv('embeddings/metadata.csv', index=False)

# Save model info
model_info = {
    'model_name': model_name,
    'embedding_dim': image_embeddings.shape[1],
    'num_samples': len(embeddings_data),
    'device_used': str(device)
}

with open('embeddings/model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print("Embeddings and metadata saved successfully!")
print(f"Model info: {model_info}")


## 7. Verify Embeddings


In [ ]:
# Load and verify saved embeddings
loaded_image_embeddings = np.load('embeddings/image_embeddings.npy')
loaded_text_embeddings = np.load('embeddings/text_embeddings.npy')
loaded_metadata = pd.read_csv('embeddings/metadata.csv')

print(f"Loaded image embeddings shape: {loaded_image_embeddings.shape}")
print(f"Loaded text embeddings shape: {loaded_text_embeddings.shape}")
print(f"Loaded metadata shape: {loaded_metadata.shape}")

# Verify embeddings are normalized
image_norms = np.linalg.norm(loaded_image_embeddings, axis=1)
text_norms = np.linalg.norm(loaded_text_embeddings, axis=1)

print(f"\nImage embedding norms - min: {image_norms.min():.6f}, max: {image_norms.max():.6f}")
print(f"Text embedding norms - min: {text_norms.min():.6f}, max: {text_norms.max():.6f}")
print("Embeddings are properly normalized!" if np.allclose(image_norms, 1.0) and np.allclose(text_norms, 1.0) else "Warning: Embeddings may not be normalized")


## 8. Test Basic Similarity


In [ ]:
# Test basic similarity between image and text embeddings
def test_similarity(image_idx, text_idx):
    """Test similarity between specific image and text embeddings"""
    img_emb = loaded_image_embeddings[image_idx]
    txt_emb = loaded_text_embeddings[text_idx]
    
    similarity = cosine_similarity([img_emb], [txt_emb])[0][0]
    
    print(f"Image: {loaded_metadata.iloc[image_idx]['image_id']}")
    print(f"Text: {loaded_metadata.iloc[text_idx]['caption']}")
    print(f"Similarity: {similarity:.4f}")
    
    return similarity

# Test a few examples
print("Testing image-text similarity:")
print("=" * 50)

for i in range(min(3, len(loaded_metadata))):
    test_similarity(i, i)  # Same image-caption pair
    print()


## 9. Summary and Next Steps


In [ ]:
print("Part 1 Complete! Summary:")
print("=" * 40)
print(f"✓ Dataset loaded: {len(captions_data)} images with captions")
print(f"✓ Model selected: {model_name}")
print(f"✓ Embeddings generated: {len(embeddings_data)} image-text pairs")
print(f"✓ Embeddings stored: {loaded_image_embeddings.shape[0]} samples")
print(f"✓ Embedding dimension: {loaded_image_embeddings.shape[1]}")
print(f"✓ Files created:")
print(f"  - embeddings/image_embeddings.npy")
print(f"  - embeddings/text_embeddings.npy")
print(f"  - embeddings/metadata.csv")
print(f"  - embeddings/model_info.json")
print("\nReady for Part 2: Search Functionality!")
